In [ ]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON

In [ ]:
# export
# Python native modules
import os,warnings
# Third party libs
from fastcore.all import *
from fastai.torch_core import *
from fastai.basics import *
import pandas as pd
import torch
import numpy as np
# Local modules

In [ ]:
# default_exp core

# Core
> Core libs for fastrl

## D
> A better dictionary

In [ ]:
# export
def map_dict_ex(d,f,*args,gen=False,wise=None,**kwargs):
    "Like `map`, but for dicts and uses `bind`, and supports `str` and indexing"
    g = (bind(f,*args,**kwargs) if callable(f)
         else f.format if isinstance(f,str)
         else f.__getitem__)

    if wise is None:  return map(g,d.items())
    return ((k,g(v)) if wise=='value' else (g(k),v) for k,v in d.items())

Check that general mapping for dicts works nicely...

In [ ]:
test_dict={'a':1,'b':2,'c':3}
test_eq(dict(map_dict_ex(test_dict,lambda t:(t[0]+'_new',t[1]+1))),{'a_new':2,'b_new':3,'c_new':4})

Check that key and value wise mapping works correctly...

In [ ]:
test_eq(dict(map_dict_ex(test_dict,lambda k:k+'_new',wise='key')),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(map_dict_ex(test_dict,lambda v:v+1,wise='value')),{'a':2,'b':3,'c':4})

In [ ]:
# export
_error_msg='Found idxs: %s have values more than %s e.g.: %s'

class D(dict):
    "Improved version of `dict` with array handling abilities"
    def __init__(self,*args,mapping=False,**kwargs):
        self.mapping=mapping
        super().__init__(*args,**kwargs)
        
    def eq_k(self,o:'D',with_diff=False):
        eq=set(o.keys())==set(self.keys())
        if with_diff: return eq,set(o.keys()).symmetric_difference(set(self.keys()))
        return eq
    def _new(self,*args,**kwargs): return type(self)(*args,**kwargs)
    
    def map(self,f,*args,gen=False,**kwargs): 
        return (self._new,noop)[gen](map_dict_ex(self,f,*args,**kwargs),mapping=True)
    def mapk(self,f,*args,gen=False,wise='key',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)
    def mapv(self,f,*args,gen=False,wise='value',**kwargs):
        return self.map(f,*args,gen=gen,wise=wise,**kwargs)

In [ ]:
test_dict=D({'a':1,'b':2,'c':3})
test_eq(test_dict.map(lambda t:(t[0]+'_new',t[1]+1)),{'a_new':2,'b_new':3,'c_new':4})
test_eq(isinstance(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True),map),True)
test_eq(dict(test_dict.map(lambda t:(t[0]+'_new',t[1]+1),gen=True)),{'a_new':2,'b_new':3,'c_new':4})

test_eq(test_dict.mapk(lambda k:k+'_new'),{'a_new':1,'b_new':2,'c_new':3})
test_eq(dict(test_dict.mapk(lambda k:k+'_new',gen=True)),{'a_new':1,'b_new':2,'c_new':3})

test_eq(test_dict.mapv(lambda v:v+1,wise='value'),{'a':2,'b':3,'c':4})
test_eq(dict(test_dict.mapv(lambda v:v+1,gen=True,wise='value')),{'a':2,'b':3,'c':4})

`BD` is the primary data structure that `fastrl` uses. It allows for easily iterating and doing operations on steps attained from environments.

## BD 
> A batch wise dictionary that requires all values to be numpy,tensor, or None.

We need to change any indexer that is passed. We don't know if the indexer is going to
be a numpy array, slice, tensor, or int.
All we know is 2 things:
- If it is an int, the batch dim will disappear
- If it is an indexer, then the batch dim will stay, but be smaller

In [ ]:
# export
def tensor2shape(k,t:'TensorBatch',relative_shape=False):
    "Converts a tensor into a dict of shapes, or a 1d numpy array"
    return {
        k:t.cpu().numpy().reshape(-1,) if len(t.shape)==2 and t.shape[1]==1 else 
        [str((1,*t.shape[1:]) if relative_shape else t.shape)]*t.shape[0]
    }

`tensor2shape` is a function for preparing tensors for showing in pandas. For example
if we have a tensor that has 5 dimensions, it would be very hard to read if displayed in pandas

In [ ]:
test_eq(tensor2shape('test',torch.randn(3,5)),
       {'test': ['torch.Size([3, 5])', 'torch.Size([3, 5])', 'torch.Size([3, 5])']})

If the tensor has only 1 channel, then we can show its literal value...

In [ ]:
test_eq(tensor2shape('test',torch.tensor([[1],[2],[3]]))['test'],
        {'test': np.array([1, 2, 3])}['test'])

In [ ]:
# export
def tensor2mu(k,t:Tensor): return {f'{k}_mu':t.reshape(t.shape[0],-1).double().mean(axis=1)}
tensor2mu.__docs__="Returns a dict with key `k`_mu with the mean of `t` batchwise "

Outputs a dictionary that has the mean of the tensor. The returned dictionary's keys 
have the naming convention: *[k]_mu*.

In [ ]:
o=torch.randn(3,5)
test_eq(tensor2mu('test',o)['test_mu'],{'test_mu': o.double().mean(axis=1)}['test_mu'])

Ok I have reworked the tensor management soooo many times. I think the core issue is the tensors themselves. They should individually be incharge of
determining if they match the expected batch size I think....

In [ ]:
# export
class TensorBatch(TensorBase):
    "A tensor that maintains and propagates a batch dimension"
    def __new__(cls, x, bs=1,**kwargs):
        res=super(TensorBatch,cls).__new__(cls,x,**kwargs)
        
        if bs==1:
            if len(res.shape)<2: 
                res=res.unsqueeze(0)
        else:
            if res.shape[0]!=bs and res.shape[1]==bs and len(res.shape)==2: 
#                 print('tansposing',res,bs)
                res=torch.transpose(res,1,0)
            
        assert len(res.shape)>1,f'Tensor has shape {res.shape} while bs is {bs}'
        return res
    
    @property
    def bs(self): return self.shape[0]
    def get(self,*args):
#         print(self)
        res=self[args]
        if len(self.shape)>len(res.shape): res=res.unsqueeze(0)
        return res
    
    @classmethod
    def vstack(cls,*args):
        new_bs=sum(map(_get_bs,*args))
        return cls(torch.vstack(*args),bs=new_bs)
              
def obj2tensor(o):
    return (o if isinstance(o,TensorBatch) else
            TensorBatch(o) if isinstance(o,(L,list,np.ndarray,Tensor,TensorBatch)) else
            TensorBatch([o])) 

def _get_bs(o): return o.bs if isinstance(o,TensorBatch) else TensorBatch(o).bs

# export
class BD(D):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        if not self.mapping: self.update(self.mapv(obj2tensor))
        self.bs=list(self.values())[0].bs

    def __radd__(self,o): return self if isinstance(o,int) else self.__add__(o) 
    def __add__(self,o):
#         print('add',self.bs)
        return BD({k:TensorBatch.vstack((self[k],o[k])) for k in self})
    
    def __getitem__(self,o):
        if is_listy(o) or isinstance(o,(TensorBatch,int,Tensor)): 
            return type(self)({k:self[k].get(o) for k in self})
        return super().__getitem__(o)

    @classmethod
    def merge(cls,*ds,**kwargs): return cls(merge(*ds),**kwargs)
    @delegates(pd.DataFrame)
    def pandas(self,mu=False,relative_shape=False,**kwargs):
        "Turns a `BD` into a pandas Dataframe optionally showing `mu` of values."
        return pd.DataFrame(merge(
            *tuple(tensor2shape(k,v,relative_shape) for k,v in self.items()),
            *(tuple(tensor2mu(k,v) for k,v in self.items()) if mu else ())
        ),**kwargs)

> Note: I think that BD should do zero undirected shae correction. I think it would be better for it to validate the shapes have batch dims
    that match. But I think that the __init__ should accept a shape_map for a key->single batch shape. I can have a default 
    key map so it can still be convenient, however this would open up BD to be more flexible.

In [ ]:
TensorBatch.vstack((Tensor([[1,2,3,4]]),Tensor([[1,2,3,4]])))

TensorBatch([[1., 2., 3., 4.],
        [1., 2., 3., 4.]])

In [ ]:
TensorBatch([1]).bs

1

Ok so the solution was that `BD` itself does not validate or coerce batch sizes.
It does not check that they all match.
It merely uses the TensorBatch object in all its operations.
The TensorBatch object tracks and manages what the batch size is really supposed to be.

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 04_callback.core.ipynb.
Converted 05_data.block.ipynb.
Converted 05_data.test_async.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/00_core.ipynb
converting: /home/fastrl_user/fastrl/nbs/05_data.block.ipynb
converting: /home/fastrl_user/fastrl/nbs/04_callback.core.ipynb
An error occurred while executing the following cell:
------------------
from nbdev.showdoc import show_doc
from fastrl.callback.core import *
------------------


ModuleNotFoundErrorTraceback (most recent call last)
<ipython-input-1-83bd0ee2f93c> in <module>
      1 from nbdev.showdoc import show_doc
----> 2 from fastrl.callback.core import *

ModuleNotFoundError: No module named 'fastrl.callback'
ModuleNotFoundError: No module named 'fastrl.callback'

An error occurred while executing the following c